# PathML ONNX Tutorial

Written by James Wen. James_Wen@dfci.harvard.edu. 

[![View on GitHub](https://img.shields.io/badge/View-on%20GitHub-lightgrey?logo=github)](https://github.com/Dana-Farber-AIOS/pathml/blob/master/examples/)

## Introduction

This notebook is a tutorial on how to use the future ONNX `inference` feature in PathML. 

Some notes:
- The ONNX inference pipeline uses the existing PathML Pipeline and Transforms infrastructure.
    - ONNX labels are saved to a `pathml.core.slide_data.SlideData` object as `tiles`.
    - Users can iterate over the tiles as they would when using this feature for preprocessing. 
- Preprocessing images before inference
    - Users will need to create their own bespoke `pathml.preprocessing.transforms.transform` method to preprocess images before inference if necessary.
    - A guide on how to create preprocessing pipelines is [here](https://pathml.readthedocs.io/en/latest/creating_pipelines.html). 
    - A guide on how to run preprocessing pipelines is [here](https://pathml.readthedocs.io/en/latest/running_pipelines.html). 
- ONNX Model Initializers 
    - ONNX models often have neural network initializers stored in the input graph. This means that the user is expected to specify initializer values when running inference. To solve this issue, we have a function that removes the network initializers from the input graph. This functions is adopted from the `onnxruntime` [github](https://github.com/microsoft/onnxruntime/blob/main/tools/python/remove_initializer_from_input.py).  
    - We also have a function that checks if the initializers have been removed from the input graph before running inference. Both of these functions are described more below. 
- When using a model stored remotely on HuggingFace, the model is *downloaded locally* before being used. The user will need to delete the model after running `Pipeline` with a method that comes with the model class. An example of how to do this is below. 

## Quick Sample Code
- Below is an example of how users would use the ONNX inference feature in PathML with a locally stored model.
```python
# load packages
from pathml.core import SlideData

from pathml.preprocessing import Pipeline
import pathml.preprocessing.transforms as Transforms

from pathml.inference import Inference, remove_initializer_from_input

# Define slide path
slide_path = 'PATH TO SLIDE'

# Set path to model 
model_path = 'PATH TO ONNX MODEL'
# Define path to export fixed model
new_path = 'PATH TO SAVE NEW ONNX MODEL'

# Fix the ONNX model by removing initializers. Save new model to `new_path`. 
remove_initializer_from_input(model_path, new_path) 

inference = Inference(model_path = new_path, input_name = 'data', num_classes = 8, model_type = 'segmentation')

# Create a transformation list
transformation_list = [
    inference
] 

# Initialize pathml.core.slide_data.SlideData object
wsi = SlideData(slide_path, stain = 'Fluor')

# Set up PathML pipeline
pipeline = Pipeline(transformation_list)

# Run Inference
wsi.run(pipeline, tile_size = 1280, level = 0)
```

- Below is an example of how users would use the ONNX inference feature in PathML with a model stored in the public HuggingFace repository.
```python
# load packages
from pathml.core import SlideData

from pathml.preprocessing import Pipeline
import pathml.preprocessing.transforms as Transforms

from pathml.inference import RemoteTestHoverNet

# Define slide path
slide_path = 'PATH TO SLIDE'

inference = RemoteTestHoverNet()

# Create a transformation list
transformation_list = [
    inference
] 

# Initialize pathml.core.slide_data.SlideData object
wsi = SlideData(slide_path)

# Set up PathML pipeline
pipeline = Pipeline(transformation_list)

# Run Inference
wsi.run(pipeline, tile_size = 256)

# DELETE ONNX MODEL DOWNLOADED FROM HUGGINGFACE
inference.remove() 
```

## Load Packages

**NOTE**
- Please put in your environment name in the following line if you are using a jupyter notebook. If not, you may remove this line. 
    `os.environ["JAVA_HOME"] = "/opt/conda/envs/YOUR ENVIRONMENET NAME"` 

In [1]:
import os
os.environ["JAVA_HOME"] = "/opt/conda/envs/YOUR ENVIRONMENET NAME" # TO DO: CHANGE THIS TO YOUR ENVIRONMENT NAME
import numpy as np 
import onnx
import onnxruntime as ort 
import requests
import torch

from pathml.core import SlideData, Tile
from dask.distributed import Client
from pathml.preprocessing import Pipeline
import pathml.preprocessing.transforms as Transforms

from pathml.inference import (
    HaloAIInference,
    Inference,
    InferenceBase,
    RemoteTestHoverNet,
    check_onnx_clean,
    remove_initializer_from_input,
)

## ONNX Inference Class and ONNX Model Fixer

- Here is the raw code for the functions that handle the initializers in the ONNX model and the classes that run the inference.

### Functions to remove initializers and check that initializers have been removed.

- `remove_initializer_from_input`
    - This function removes any initializers from the input graph of the ONNX model.
    - Without removing the initializers from the input graph, users will not be able to run inference.
    - Adapted from the `onnxruntime` [github](https://github.com/microsoft/onnxruntime/blob/main/tools/python/remove_initializer_from_input.py).  
    - Users specify:
        - `model_path` (str): path to ONNX model,
        - `new_path` (str): path to save adjusted model w/o initializers
    - We will run this function on all models placed in our model zoo, so users will not have to run it unless they are working with their own local models.
    
 <br> 
 
- `check_onnx_clean`
    - Checks if the initializers are in the input graph
    - Returns `True` and a `ValueError` if there are initializers in the input graph
    - Adapted from the `onnxruntime` [github](https://github.com/microsoft/onnxruntime/blob/main/tools/python/remove_initializer_from_input.py). 
    - Users specify:
        - `model_path` (str): path to ONNX model

 <br> 

 - `convert_pytorch_onnx` 
    - Converts a PyTorch `.pt` file to `.onnx`
    - Wrapper function of the [PyTorch](https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html) function to handle the conversion.
    - Users specify:
        - model_path (torch.nn.Module Model): Pytorch model to be converted,
        - dummy_tensor (torch.tensor): dummy input tensor that is an example of what will be passed into the model,
        - model_name (str): name of ONNX model created with .onnx at the end,
        - opset_version (int): which opset version you want to use to export
        - input_name (str): name assigned to dummy_tensor
    - Note that the model class must be defined before loading the `.pt` file and set to eval before calling this function. 

### Inference Classes

<br> 

- `InferenceBase`
    - This class inherits from `pathml.preprocessing.transforms.transform`, similar to all of the preprocessing transformations. Inheriting from `transforms.transform` allows us to use the existing `Pipeline` function in PathML which users should be familar with.  
    - This is the base class for all Inference classes for ONNX modeling
    - Each instance of a class also comes with a `model_card` which specifies certain details of the model in dictionary form. The default parameters are:
        -   ```python 
                self.model_card = {
                'name' : None, 
                'num_classes' : None,
                'model_type' : None, 
                'notes' : None,  
                'model_input_notes': None, 
                'model_output_notes' : None,
                'citation': None } 
            ``` 
       - Model cards are where important information about the model should be kept. Since they are in dictionary form, the user can add keys and values as they see fit. 
       - This class also has getter and setter functions to adjust the `model_card`. Certain functions include `get_model_card`, `set_name`, `set_num_classes`, etc. 
 
  <br> 
  
- `Inference` 
    - This class is for when the user wants to use an ONNX model stored locally. 
    - Calls the `check_onnx_clean` function to check if the model is clean.
    - Users specify:
        - `model_path` (str): path to ONNX model,
        - `input_name` (str): name of input for ONNX model, *defaults to `data`* 
        - `num_classes` (int): number of outcome classes, 
        - `model_type` (str): type of model (classification, segmentation) 
        - `local` (bool): if you are using a local model or a remote model, *defaults to `True`* 
 
  <br> 
  
- `HaloAIInference`
    - This class inherits from `Inference`
    - HaloAI ONNX models always return 20 prediction maps: this class will subset and return the necessary ones. 

<br> 

- `RemoteTestHoverNet` 
    - This class inherits from `Inference` and is the test class for public models hosted on `HuggingFace`. 
    - `local` is automatically set to `False` 
    - Our current test model is a HoverNet from [TIAToolbox](https://github.com/TissueImageAnalytics/tiatoolbox)
    - Pocock J, Graham S, Vu QD, Jahanifar M, Deshpande S, Hadjigeorghiou G, Shephard A, Bashir RM, Bilal M, Lu W, Epstein D. TIAToolbox as an end-to-end library for advanced tissue image analytics. Communications medicine. 2022 Sep 24;2(1):120.
    - Its `model_card` is:
        -   ```python 
                {'name': 'Tiabox HoverNet Test',
                 'num_classes': 5,
                 'model_type': 'Segmentation',
                 'notes': None,
                 'model_input_notes': 'Accepts tiles of 256 x 256',
                 'model_output_notes': None,
                 'citation': 'Pocock J, Graham S, Vu QD, Jahanifar M, Deshpande S, Hadjigeorghiou G, Shephard A, Bashir RM, Bilal M, Lu W, Epstein D. TIAToolbox as an end-to-end library for advanced tissue image analytics. Communications medicine. 2022 Sep 24;2(1):120.'}
             ```
     
### Raw Code

Below is the raw code for your convenience. You can also find the raw code on our github. 
[![View on GitHub](https://img.shields.io/badge/View-on%20GitHub-lightgrey?logo=github)](https://github.com/Dana-Farber-AIOS/pathml/tree/master/pathml)

In [2]:
def remove_initializer_from_input(model_path, new_path):
    """Removes initializers from HaloAI ONNX models
    Taken from https://github.com/microsoft/onnxruntime/blob/main/tools/python/remove_initializer_from_input.py

    Args:
        model_path (str): path to ONNX model,
        new_path (str): path to save adjusted model w/o initializers,

    Returns:
        ONNX model w/o initializers to run inference using PathML
    """

    model = onnx.load(model_path)

    inputs = model.graph.input
    name_to_input = {}
    for onnx_input in inputs:
        name_to_input[onnx_input.name] = onnx_input

    for initializer in model.graph.initializer:
        if initializer.name in name_to_input:
            inputs.remove(name_to_input[initializer.name])

    onnx.save(model, new_path)


def check_onnx_clean(model_path):
    """Checks if the model has had it's initalizers removed from input graph.
    Adapted from from https://github.com/microsoft/onnxruntime/blob/main/tools/python/remove_initializer_from_input.py

    Args:
        model_path (str): path to ONNX model,

    Returns:
        Boolean if there are initializers in input graph.
    """

    model = onnx.load(model_path)

    inputs = model.graph.input
    name_to_input = {}
    for onnx_input in inputs:
        name_to_input[onnx_input.name] = onnx_input

    for initializer in model.graph.initializer:
        if initializer.name in name_to_input:
            return True


def convert_pytorch_onnx(
    model, dummy_tensor, model_name, opset_version=10, input_name="data"
):
    """Converts a Pytorch Model to ONNX
    Adjusted from https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html

    You need to define the model class and load the weights before exporting. See URL above for full steps.

    Args:
        model_path (torch.nn.Module Model): Pytorch model to be converted,
        dummy_tensor (torch.tensor): dummy input tensor that is an example of what will be passed into the model,
        model_name (str): name of ONNX model created with .onnx at the end,
        opset_version (int): which opset version you want to use to export
        input_name (str): name assigned to dummy_tensor

    Returns:
        Exports ONNX model converted from Pytorch
    """

    if not isinstance(model, torch.nn.Module):
        raise ValueError(
            f"The model is not of type torch.nn.Module. Received {type(model)}."
        )

    if not torch.is_tensor(dummy_tensor):
        raise ValueError(
            f"The dummy tensor needs to be a torch tensor. Received {type(dummy_tensor)}."
        )

    torch.onnx.export(
        model,
        dummy_tensor,
        model_name,
        export_params=True,
        opset_version=opset_version,
        do_constant_folding=True,
        input_names=[input_name],
    )


# Base class
class InferenceBase(Transforms.Transform):
    """
    Base class for all ONNX Models.
    Each transform must operate on a Tile.
    """

    def __init__(self):
        self.model_card = {
            "name": None,
            "num_classes": None,
            "model_type": None,
            "notes": None,
            "model_input_notes": None,
            "model_output_notes": None,
            "citation": None,
        }

    def __repr__(self):
        return "Base class for all ONNX models"

    def get_model_card(self):
        return self.model_card

    def set_name(self, name):
        self.model_card["name"] = name

    def set_num_classes(self, num):
        self.model_card["num_classes"] = num

    def set_model_type(self, model_type):
        self.model_card["model_type"] = model_type

    def set_notes(self, note):
        self.model_card["notes"] = note

    def set_model_input_notes(self, note):
        self.model_card["model_input_notes"] = note

    def set_model_output_notes(self, note):
        self.model_card["model_output_notes"] = note

    def set_citation(self, citation):
        self.model_card["citation"] = citation

    def reshape(self, image):
        """standard reshaping of tile image"""
        # flip dimensions
        # follows convention used here https://github.com/Dana-Farber-AIOS/pathml/blob/master/pathml/ml/dataset.py

        if image.ndim == 3:
            # swap axes from HWC to CHW
            image = image.transpose(2, 0, 1)
            # add a dimesion bc onnx models usually have batch size as first dim: e.g. (1, channel, height, width)
            image = np.expand_dims(image, axis=0)

            return image
        else:
            # in this case, we assume that we have XYZCT channel order
            # so we swap axes to TCZYX for batching
            # note we are not adding a dim here for batch bc we assume that subsetting will create a batch "placeholder" dim
            image = image.T

            return image

    def F(self, target):
        """functional implementation"""
        raise NotImplementedError

    def apply(self, tile):
        """modify Tile object in-place"""
        raise NotImplementedError


# class to handle local onnx models
class Inference(InferenceBase):
    """Transformation to run inferrence on ONNX model.

    Assumptions:
        - The ONNX model has been cleaned by `remove_initializer_from_input` first

    Args:
        model_path (str): path to ONNX model w/o initializers,
        input_name (str): name of the input the ONNX model accepts
    """

    def __init__(
        self,
        model_path=None,
        input_name="data",
        num_classes=None,
        model_type=None,
        local=True,
    ):
        super().__init__()

        self.input_name = input_name
        self.num_classes = num_classes
        self.model_type = model_type
        self.local = local

        if self.local:
            # using a local onnx model
            self.model_path = model_path
        else:
            # if using a model from the model zoo, set the local path to a temp file
            self.model_path = "temp.onnx"

        # fill in parts of the model_card with the following info
        self.model_card["num_classes"] = self.num_classes
        self.model_card["model_type"] = self.model_type

        # check if there are initializers in input graph if using a local model
        if local:
            if check_onnx_clean(model_path):
                raise ValueError(
                    "The ONNX model still has graph initializers in the input graph. Use `remove_initializer_from_input` to remove them."
                )
        else:
            pass

    def __repr__(self):
        if self.local:
            return f"Class to handle ONNX model locally stored at {self.model_path}"
        else:
            return f"Class to handle a {self.model_card['model_name']} from the PathML model zoo."

    def inference(self, image):
        # reshape the image
        image = self.reshape(image)

        # load fixed model
        onnx_model = onnx.load(self.model_path)

        # check tile dimensions match ONNX input dimensions
        input_node = onnx_model.graph.input

        dimensions = []
        for input in input_node:
            if input.name == self.input_name:
                input_shape = input.type.tensor_type.shape.dim
                for dim in input_shape:
                    dimensions.append(dim.dim_value)

        assert (
            image.shape[-1] == dimensions[-1] and image.shape[-2] == dimensions[-2]
        ), f"expecting tile shape of {dimensions[-2]} by {dimensions[-1]}, got {image.shape[-2]} by {image.shape[-1]}"

        # check onnx model
        onnx.checker.check_model(onnx_model)

        # start an inference session
        ort_sess = onnxruntime.InferenceSession(self.model_path)

        # create model output, returns a list
        model_output = ort_sess.run(None, {self.input_name: image.astype("f")})

        return model_output

    def F(self, image):
        # run inference function
        prediction_map = self.inference(image)

        # single task model
        if len(prediction_map) == 1:
            # return first and only prediction array in the list
            return prediction_map[0]

        # multi task model
        else:
            # concatenate prediction results
            # assumes that the tasks all output prediction arrays of same dimension on H and W
            result_array = np.concatenate(prediction_map, axis=1)
            return result_array

    def apply(self, tile):
        tile.image = self.F(tile.image)


class HaloAIInference(Inference):
    """Transformation to run inferrence on HALO AI ONNX model.

    Assumptions:
        - Assumes that the ONNX model returns a tensor in which there is one prediction map for each class
        - For example, if there are 5 classes, the ONNX model will output a (1, 5, Height, Weight) tensor
        - If you select to argmax the classes, the class assumes a softmax or sigmoid has already been applied
        - HaloAI ONNX models always have 20 class maps so you need to index into the first x maps if you have x classes


    Args:
        model_path (str): path to ONNX model w/o initializers,
        num_classes (int): number of classes in the data,
        input_name (str): name of the input the ONNX model accepts
    """

    def __init__(
        self,
        model_path=None,
        input_name="data",
        num_classes=None,
        model_type=None,
        local=True,
    ):
        super().__init__(model_path, input_name, num_classes, model_type, local)

        self.model_card["num_classes"] = self.num_classes
        self.model_card["model_type"] = self.model_type

    def __repr__(self):
        return f"Class to handle HALO AI ONNX model locally stored at {self.model_path}"

    def F(self, image):
        prediction_map = self.inference(image)

        prediction_map = prediction_map[0][:, 0 : self.num_classes, :, :]

        return prediction_map

    def apply(self, tile):
        tile.image = self.F(tile.image)


# class to handle remote onnx models
class RemoteTestHoverNet(Inference):
    """Transformation to run inferrence on ONNX model.

    Citation for model:
    Pocock J, Graham S, Vu QD, Jahanifar M, Deshpande S, Hadjigeorghiou G, Shephard A, Bashir RM, Bilal M, Lu W, Epstein D.
    TIAToolbox as an end-to-end library for advanced tissue image analytics. Communications medicine. 2022 Sep 24;2(1):120.

    Args:
        model_path (str): temp file name to download onnx from huggingface,
        input_name (str): name of the input the ONNX model accepts
    """

    def __init__(
        self,
        model_path="temp.onnx",
        input_name="data",
        num_classes=5,
        model_type="Segmentation",
        local=False,
    ):
        super().__init__(model_path, input_name, num_classes, model_type, local)

        # specify URL of the model in PathML public repository
        url = "https://huggingface.co/pathml/test/resolve/main/hovernet_fast_tiatoolbox_fixed.onnx"

        # download model, save as temp.onnx
        with open(self.model_path, "wb") as out_file:
            content = requests.get(url, stream=True).content
            out_file.write(content)

        self.model_card["num_classes"] = self.num_classes
        self.model_card["model_type"] = self.model_type
        self.model_card["name"] = "Tiabox HoverNet Test"
        self.model_card["model_input_notes"] = "Accepts tiles of 256 x 256"
        self.model_card[
            "citation"
        ] = "Pocock J, Graham S, Vu QD, Jahanifar M, Deshpande S, Hadjigeorghiou G, Shephard A, Bashir RM, Bilal M, Lu W, Epstein D. TIAToolbox as an end-to-end library for advanced tissue image analytics. Communications medicine. 2022 Sep 24;2(1):120."

    def __repr__(self):
        return "Class to handle remote TIAToolBox HoverNet test ONNX. See model card for citation."

    def apply(self, tile):
        tile.image = self.F(tile.image)

    def remove(self):
        # remove the temp.onnx model
        os.remove(self.model_path)


## Try it Yourself!

- What you need:
    - An ONNX model stored locally
    - An image with which you want to run inference stored locally
    - PathML already downloaded 

- Make sure to define the `Inference` class and `remove_initializer_from_input` above in the previous seciton if you have not downloaded the latest version of PathML.

- You will need to define the following variables: 
    - `slide_path`: 'PATH TO SLIDE'
    - `model_path`: 'PATH TO ONNX MODEL'
    - `new_path`: 'PATH TO SAVE FIXED ONNX MODEL'
    - `num_classes`: 'NUMBER OF CLASSES IN YOUR DATASET'
    - `tile_size`: 'TILE SIZE THAT YOUR ONNX MODEL ACCEPTS'
    
- The code in the cell below assumes you want the images passed in as is. If you need to select channels, you will need to add another `transform` method to do so before the inference transform. The following code provides an example if you want to subset into the first channel of an image. *Remember that PathML reads images in as XYZCT.* 

```python 
class convert_format(Transforms.Transform):
    def F(self, image):
        # orig = (1280, 1280, 1, 6, 1) = (XYZCT)
        image = image[:, :, :, 0, ...] # this will make the tile (1280, 1280, 1, 1)
        return image

    def apply(self, tile):
        tile.image = self.F(tile.image)
        
convert = convert_format()
inference = Inference(
    model_path = 'PATH TO LOCAL MODEL', 
    input_name = 'data', 
    num_classes = 'NUMBER OF CLASSES' , 
    model_type = 'CLASSIFICATION OR SEGMENTATION', 
    local = True)

transformation_list = [convert, inference] 

```

### Converting a Pytorch Model to ONNX Using the `convert_pytorch_onnx` Function

Note the following:
- Similar to PyTorch, you will need to define and create an instance of you model class before loading the `.pt` file. Then you will need to set it to eval mode before calling the conversion function. The code to do these steps is below.

In [ ]:
# Define your model class
num_input, num_output, batch_size = 10, 1, 1

class SimpleModel(torch.nn.Module):
  def __init__(self):
    super(SimpleModel, self).__init__()
    self.linear = torch.nn.Linear(num_input, num_output)
    torch.nn.init.xavier_uniform_(self.linear.weight)
  def forward(self, x):
    y = self.linear(x)
    return y

# Define your model var
model = SimpleModel()

# Export model as .pt if you haven't already done so
# If you have already exported a .pt file, you will still need to define a model class, initialize it, and set it to eval mode. 
# If you saved your model using `torch.jit.script`, you will not need to define your model class and instead load it using `torch.jit.load` then set it to eval mode.
torch.save(model, "test.pt")

# Load .pt file
model_test = torch.load("test.pt")
# Set model to eval mode
model_test.eval()

# Define a dummy tensor (this is an example of what the ONNX should expect during inference)
x = torch.randn(batch_size, num_input)

# Run conversion function
convert_pytorch_onnx(model = model_test, dummy_tensor = x, model_name = "NAME_OF_OUTPUT_MODEL_HERE.onnx")

### Local ONNX Model Using the `Inference` Class

In [ ]:
# Define slide path
slide_path = 'PATH TO SLIDE'

# Set path to model 
model_path = 'PATH TO ONNX MODEL'
# Define path to export fixed model
new_path = 'PATH TO SAVE NEW ONNX MODEL'


# Fix the ONNX model
remove_initializer_from_input(model_path, new_path) 

inference = Inference(model_path = new_path, input_name = 'data', num_classes = 'NUMBER OF CLASSES' , model_type = 'CLASSIFICATION OR SEGMENTATION', local = True)

transformation_list = [inference] 

# Initialize pathml.core.slide_data.SlideData object
wsi = SlideData(slide_path)

# Set up PathML pipeline
pipeline = Pipeline(transformation_list)

# Run Inference
# Level is equal to 0 for highest resolution (Note that this is the default setting)
wsi.run(pipeline, tile_size = 'TILE SIZE THAT YOUR ONNX MODEL ACCEPTS', level = 0)

### Local ONNX Model Using the `HaloAIInference` Class

In [ ]:
# Define slide path
slide_path = 'PATH TO SLIDE'

# Set path to model 
model_path = 'PATH TO ONNX MODEL'
# Define path to export fixed model
new_path = 'PATH TO SAVE NEW ONNX MODEL'


# Fix the ONNX model
remove_initializer_from_input(model_path, new_path) 

inference = HaloAIInference(model_path = new_path, input_name = 'data', num_classes = 'NUMBER OF CLASSES' , model_type = 'CLASSIFICATION OR SEGMENTATION', local = True)

transformation_list = [inference] 

# Initialize pathml.core.slide_data.SlideData object
wsi = SlideData(slide_path)

# Set up PathML pipeline
pipeline = Pipeline(transformation_list)

# Run Inference
# Level is equal to 0 for highest resolution (Note that this is the default setting)
wsi.run(pipeline, tile_size = 'TILE SIZE THAT YOUR ONNX MODEL ACCEPTS', level = 0)

### Remote ONNX Using our `RemoteTestHoverNet` Class
- Uses a Hovernet from [TIAToolbox](https://github.com/TissueImageAnalytics/tiatoolbox) 
- Note that the purpose of this model is to illustrate how PathML will handle future remote models. We plan on release more public models to our model zoo on HuggingFace in the future.
- Citation for model:
    - Pocock J, Graham S, Vu QD, Jahanifar M, Deshpande S, Hadjigeorghiou G, Shephard A, Bashir RM, Bilal M, Lu W, Epstein D. TIAToolbox as an end-to-end library for advanced tissue image analytics. Communications medicine. 2022 Sep 24;2(1):120.
- Make sure your image has 3 channels! 
- When the `RemoteTestHoverNet` is first initialized, it downloads the HoverNet from HuggingFace and saves it locally on your own system as `temp.onnx`. 
    - **You will need to remove it manually by calling the `remove()` method** An example of how to call this method is in the last line in the code below. 

In [ ]:
# Define slide path
slide_path = 'PATH TO SLIDE'

inference = RemoteTestHoverNet()

# Create a transformation list
transformation_list = [
    inference
] 

# Initialize pathml.core.slide_data.SlideData object
wsi = SlideData(slide_path)

# Set up PathML pipeline
pipeline = Pipeline(transformation_list)

# Run Inference
wsi.run(pipeline, tile_size = 256)

# DELETE ONNX MODEL DOWNLOADED FROM HUGGINGFACE
inference.remove() 

## Iterate over the tiles

Now that you have your tiles saved to your SlideData object, you can now iterate over them.

For example, if you wanted to check the shape of the tiles you could run the following code: 

```python
for tile in wsi.tiles: 
    print(tile.image.shape) 
```

## References

- Pocock J, Graham S, Vu QD, Jahanifar M, Deshpande S, Hadjigeorghiou G, Shephard A, Bashir RM, Bilal M, Lu W, Epstein D. TIAToolbox as an end-to-end library for advanced tissue image analytics. Communications medicine. 2022 Sep 24;2(1):120.

- https://github.com/microsoft/onnxruntime/blob/main/tools/python/remove_initializer_from_input.py